In [1]:
import numpy.random
import matplotlib.pyplot as plt
from kendall import KendallSimulator, KendallServer, KendallEvent
from kendall.queues import RandomServerQueue
from kendall.spawners import ExponentialSpawner


def average(list):
    return sum(list) / float(len(list))


class TelephoneLine(KendallServer):
    def __init__(self, queue, id):
        self.queue = queue
        self.id = id
        self.reset()

    def assign(self, event, time):
        self.event = event
        self.start_time = time

    def complete(self, event, time):
        self.event = None
        self.total_time += time - self.start_time
        self.start_time = None

    def reset(self):
        self.event = None
        self.start_time = None
        self.total_time = 0.00


class CallEvent(KendallEvent):
    average_call_duration = 300.0
    count = 0
    dropped = 0
    last_spawn_time = 0.0
    spawn_delays = []
    call_durations = []
    all_calls = []

    def __init__(self, *args, **kwargs):
        super(CallEvent, self).__init__(*args, **kwargs)
        CallEvent.count += 1
        CallEvent.all_calls.append(self)
        self.count = CallEvent.count
        self._processing_time = numpy.random.exponential(CallEvent.average_call_duration)

    def on_enqueue(self, queue, time):
        self.enter_time = time
        CallEvent.spawn_delays.append(time - CallEvent.last_spawn_time)
        CallEvent.call_durations.append(self._processing_time)
        CallEvent.last_spawn_time = time
        # print "Ship {} entered harbor at {}! Current queue: {}".format(self.count, time, len(queue.queue))

    def on_drop(self, queue, time):
        CallEvent.dropped += 1
        self.status = "dropped"
        # print "Call {} was dropped at {}!".format(self.count, time)

    def on_process(self, server, time):
        # print "Call {} was started at {}!".format(self.count, time)
        self.status = "processed"
        pass

    def on_finish(self, queue, time):
        self.exit_time = time
        # print "Call {} was finished at {}!".format(self.count, time)

    def processing_time(self, server):
        return self._processing_time


class CallEventAtoB(CallEvent):
    pass


class CallEventBtoA(CallEvent):
    pass


class TelephoneSystem(RandomServerQueue):
    server_class = TelephoneLine


class CallSpawnerAtoB(ExponentialSpawner):
    pipe_event_class = CallEventAtoB

class CallSpawnerBtoA(ExponentialSpawner):
    pipe_event_class = CallEventBtoA


class AsymmetricTelephonySimulator(KendallSimulator):
    def __init__(self, time_limit, a_to_b_lines, b_to_a_lines):
        super(AsymmetricTelephonySimulator, self).__init__(time_limit)

        self.city_a_spawner = CallSpawnerAtoB(self, 12.0)
        self.city_b_spawner = CallSpawnerBtoA(self, 15.0)
        self.a_to_b_system = TelephoneSystem(self, server_count=a_to_b_lines, max_queued=0)
        self.b_to_a_system = TelephoneSystem(self, server_count=b_to_a_lines, max_queued=0)

        self.register_queue("City A Spawner", self.city_a_spawner)
        self.register_queue("City B Spawner", self.city_b_spawner)
        self.register_queue("A to B System", self.a_to_b_system)
        self.register_queue("B to A System", self.b_to_a_system)
        self.city_a_spawner.pipe(self.a_to_b_system)
        self.city_b_spawner.pipe(self.b_to_a_system)

        self.all_calls = []
        self.all_spawn_delays = []
        self.all_call_durations = []
        self.all_dropped_percentages = []
        self.all_minimums = []
        self.all_averages = []
        self.all_maximums = []
        self.all_a_to_b_utilizations = []
        self.all_b_to_a_utilizations = []

    def reset(self):
        super(AsymmetricTelephonySimulator, self).reset()
        self.city_a_spawner.start()
        self.city_b_spawner.start()
        CallEvent.count = 0
        CallEvent.dropped = 0
        CallEvent.last_spawn_time = 0.0
        CallEvent.all_calls = []
        CallEvent.spawn_delays = []
        CallEvent.call_durations = []

    def collect(self):
        minimum = min(CallEvent.call_durations)
        average_d = sum(CallEvent.call_durations) / len(CallEvent.call_durations)
        maximum = max(CallEvent.call_durations)

        self.all_calls.append(CallEvent.all_calls[:])
        self.all_spawn_delays.extend(CallEvent.spawn_delays)
        self.all_call_durations.extend(CallEvent.call_durations)
        self.all_minimums.append(minimum)
        self.all_maximums.append(maximum)
        self.all_averages.append(average_d)
        self.all_dropped_percentages.append(CallEvent.dropped / float(CallEvent.count))

        a_to_b_utilization = [s.total_time / float(TIME_LIMIT) for s in self.a_to_b_system.servers]
        b_to_a_utilization = [s.total_time / float(TIME_LIMIT) for s in self.b_to_a_system.servers]
        self.all_a_to_b_utilizations.append(average(a_to_b_utilization))
        self.all_b_to_a_utilizations.append(average(b_to_a_utilization))

class SymmetricTelephonySimulator(KendallSimulator):
    def __init__(self, time_limit, total_lines):
        super(SymmetricTelephonySimulator, self).__init__(time_limit)

        self.city_a_spawner = CallSpawnerAtoB(self, 12.0)
        self.city_b_spawner = CallSpawnerBtoA(self, 15.0)
        self.symmetric_system = TelephoneSystem(self, server_count=total_lines, max_queued=0)

        self.register_queue("City A Spawner", self.city_a_spawner)
        self.register_queue("City B Spawner", self.city_b_spawner)
        self.register_queue("Symmetric System", self.symmetric_system)
        self.city_a_spawner.pipe(self.symmetric_system)
        self.city_b_spawner.pipe(self.symmetric_system)

        self.all_calls = []
        self.all_spawn_delays = []
        self.all_call_durations = []
        self.all_dropped_percentages = []
        self.all_minimums = []
        self.all_averages = []
        self.all_maximums = []
        self.all_utilizations = []

    def reset(self):
        super(SymmetricTelephonySimulator, self).reset()
        self.city_a_spawner.start()
        self.city_b_spawner.start()
        CallEvent.count = 0
        CallEvent.dropped = 0
        CallEvent.last_spawn_time = 0.0
        CallEvent.all_calls = []
        CallEvent.spawn_delays = []
        CallEvent.call_durations = []

    def collect(self):
        minimum = min(CallEvent.call_durations)
        average_d = sum(CallEvent.call_durations) / len(CallEvent.call_durations)
        maximum = max(CallEvent.call_durations)

        self.all_calls.append(CallEvent.all_calls[:])
        self.all_spawn_delays.extend(CallEvent.spawn_delays)
        self.all_call_durations.extend(CallEvent.call_durations)
        self.all_minimums.append(minimum)
        self.all_maximums.append(maximum)
        self.all_averages.append(average_d)
        self.all_dropped_percentages.append(CallEvent.dropped / float(CallEvent.count))

        utilizations = [s.total_time / float(TIME_LIMIT) for s in self.symmetric_system.servers]
        self.all_utilizations.append(average(utilizations))


In [2]:
def generic_output(sim):
    plt.title("Number of calls made:")
    plt.hist([len(l) for l in sim.all_calls], bins='auto', edgecolor="blue")
    plt.show()

    a_to_b_calls = [[x for x in l if isinstance(x, CallEventAtoB)] for l in sim.all_calls]
    plt.title("Number of calls made from A to B:")
    plt.hist(map(len, a_to_b_calls), bins='auto', edgecolor="blue")
    plt.show()
    print "Average calls made from A to B:", average(map(len, a_to_b_calls))

    b_to_a_calls = [[x for x in l if isinstance(x, CallEventBtoA)] for l in sim.all_calls]
    plt.title("Number of calls made from B to A:")
    plt.hist(map(len, b_to_a_calls), bins='auto', edgecolor="blue")
    plt.show()
    print "Average calls made from B to A:", average(map(len, b_to_a_calls))

    dropped_a_to_b_calls = [len([c for c in l if c.status == "dropped"]) for l in a_to_b_calls]
    plt.title("Dropped calls from A to B:")
    plt.hist(dropped_a_to_b_calls, bins='auto', edgecolor="blue")
    plt.show()
    print "Average number of dropped calls from A to B:", average(dropped_a_to_b_calls)

    dropped_b_to_a_calls = [len([c for c in l if c.status == "dropped"]) for l in b_to_a_calls]
    plt.title("Dropped calls from B to A:")
    plt.hist(dropped_b_to_a_calls, bins='auto', edgecolor="blue")
    plt.show()
    print "Average number of dropped calls B to A:", average(dropped_b_to_a_calls)

    
def test_asymmetric(runs=1000, time_limit=3000, a_to_b=10, b_to_a=10):
    sim = AsymmetricTelephonySimulator(time_limit, a_to_b, b_to_a)
    for x in xrange(0, runs):
        sim.reset()
        sim.run()
        sim.collect()

    plt.title("Percent dropped calls with {} X to Y and {} Y to X lines".format(a_to_b, b_to_a))
    plt.hist(sim.all_dropped_percentages, range=[0.0, 1.0], bins='auto', edgecolor="blue")
    plt.show()

    print "Average call drop percentage:", average(sim.all_dropped_percentages)

    good_runs = [x for x in sim.all_dropped_percentages if x < 0.05]
    print "Percent call drop percentage below 5%: {}".format(len(good_runs) / float(runs))

    generic_output(sim)

    plt.title("Average utilization of X to Y lines")
    plt.hist(sim.all_a_to_b_utilizations, range=[0.0, 1.0], bins='auto', edgecolor="blue")
    plt.show()

    print "Overall average utilization of X to Y lines:", average(sim.all_a_to_b_utilizations)

    plt.title("Average utilization of Y to X lines")
    plt.hist(sim.all_b_to_a_utilizations, range=[0.0, 1.0], bins='auto', edgecolor="blue")
    plt.show()

    print "Overall average utilization of Y to X lines:", average(sim.all_b_to_a_utilizations)

    
def test_symmetric(runs=1000, time_limit=3000, total_lines=10):
    sim = SymmetricTelephonySimulator(time_limit, total_lines)
    for x in xrange(0, runs):
        sim.reset()
        sim.run()
        sim.collect()

    plt.title("Percent dropped calls with {} symmetric lines".format(total_lines))
    plt.hist(sim.all_dropped_percentages, range=[0.0, 1.0], bins='auto', edgecolor="blue")
    plt.show()

    print "Average call drop percentage:", average(sim.all_dropped_percentages)

    good_runs = [x for x in sim.all_dropped_percentages if x < 0.05]
    print "Percent call drop percentage below 5%: {}".format(len(good_runs) / float(runs))

    generic_output(sim)

    plt.title("Average utilization of lines")
    plt.hist(sim.all_utilizations, range=[0.0, 1.0], bins='auto', edgecolor="blue")
    plt.show()

    print "Overall average utilization of lines:", average(sim.all_utilizations)


In [ ]:
TIME_LIMIT = 30000
RUN_COUNT = 500

In [ ]:
test_asymmetric(runs=RUN_COUNT, time_limit=TIME_LIMIT, a_to_b=10, b_to_a=10)

In [ ]:
test_asymmetric(runs=RUN_COUNT, time_limit=TIME_LIMIT, a_to_b=20, b_to_a=20)

In [ ]:
test_asymmetric(runs=RUN_COUNT, time_limit=TIME_LIMIT, a_to_b=25, b_to_a=25)

In [ ]:
test_asymmetric(runs=RUN_COUNT, time_limit=TIME_LIMIT, a_to_b=30, b_to_a=30)

In [ ]:
test_asymmetric(runs=RUN_COUNT, time_limit=TIME_LIMIT, a_to_b=31, b_to_a=31)

In [ ]:
test_asymmetric(runs=RUN_COUNT, time_limit=TIME_LIMIT, a_to_b=32, b_to_a=32)

In [ ]:
test_asymmetric(runs=RUN_COUNT, time_limit=TIME_LIMIT, a_to_b=33, b_to_a=33)

In [ ]:
test_asymmetric(runs=RUN_COUNT, time_limit=TIME_LIMIT, a_to_b=34, b_to_a=34)

In [ ]:
test_asymmetric(runs=RUN_COUNT, time_limit=TIME_LIMIT, a_to_b=35, b_to_a=35)

In [ ]:
test_asymmetric(runs=RUN_COUNT, time_limit=TIME_LIMIT, a_to_b=35, b_to_a=25)

In [ ]:
test_asymmetric(runs=RUN_COUNT, time_limit=TIME_LIMIT, a_to_b=35, b_to_a=28)

In [ ]:
test_asymmetric(runs=RUN_COUNT, time_limit=TIME_LIMIT, a_to_b=35, b_to_a=29)

In [ ]:
test_asymmetric(runs=RUN_COUNT, time_limit=TIME_LIMIT, a_to_b=35, b_to_a=30)

In [ ]:
test_asymmetric(runs=RUN_COUNT, time_limit=TIME_LIMIT, a_to_b=35, b_to_a=31)

In [ ]:
test_asymmetric(runs=RUN_COUNT, time_limit=TIME_LIMIT, a_to_b=35, b_to_a=32)

In [ ]:
test_symmetric(runs=RUN_COUNT, time_limit=TIME_LIMIT, total_lines=20)

In [ ]:
test_symmetric(runs=RUN_COUNT, time_limit=TIME_LIMIT, total_lines=40)

In [ ]:
test_symmetric(runs=RUN_COUNT, time_limit=TIME_LIMIT, total_lines=50)

In [ ]:
test_symmetric(runs=RUN_COUNT, time_limit=TIME_LIMIT, total_lines=55)

In [ ]:
test_symmetric(runs=RUN_COUNT, time_limit=TIME_LIMIT, total_lines=58)

In [ ]:
test_symmetric(runs=RUN_COUNT, time_limit=TIME_LIMIT, total_lines=59)

In [ ]:
test_symmetric(runs=RUN_COUNT, time_limit=TIME_LIMIT, total_lines=60)

In [ ]:
test_symmetric(runs=RUN_COUNT, time_limit=TIME_LIMIT, total_lines=51)

In [ ]:
test_asymmetric(runs=RUN_COUNT, time_limit=TIME_LIMIT, a_to_b=29, b_to_a=29)

In [ ]:
test_asymmetric(runs=RUN_COUNT, time_limit=TIME_LIMIT, a_to_b=31, b_to_a=27)